### Agents
Agents in essence are LLMs that accomplish a specific task. They can be supplemented with tools, structured output, and handoff to other agents.

In [1]:
%pip install openai-agents python-dotenv -qU

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated pa

In [9]:
%pip install eval_type_backport -qU

You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [10]:
from dotenv import load_dotenv
import os

# loads the .env file (if you have a global environment variable, you can skip this)
load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")

In [11]:
from agents import Agent, Runner

agent = Agent(
    name="Basic Agent",
    instructions="You are a helpful assistant. Respond on in all caps.",
    model="gpt-4o-mini"
)

result = await Runner.run(agent, "Hello! How are you?")
result.final_output

"HELLO! I'M DOING GREAT, THANK YOU! HOW ABOUT YOU?"

In [12]:
joke_agent = Agent(
    name="Joke Agent",
    instructions="You are a joke teller. You are given a topic and you need to tell a joke about it.",
)

topic = "Roads"
result = await Runner.run(joke_agent, topic)
result.final_output

'Why did the road join the gym?\n\nBecause it wanted to get less asphalt!'

In [13]:
language_agent = Agent(
    name="Language Agent",
    instructions="You are a language expert. You are given a joke and you need to rewrite it in a different language.",
)

joke_result = await Runner.run(joke_agent, topic)
translated_result = await Runner.run(language_agent, f"Translate this joke to Spanish: {joke_result.final_output}")
print(f"Original joke:\n{joke_result.final_output}\n")
print(f"Translated joke:\n{translated_result.final_output}")

Original joke:
Why did the road get so stressed out?

Because it had too many crossroads in life!

Translated joke:
¿Por qué la carretera estaba tan estresada?

¡Porque tenía demasiados cruces en la vida!


### Structured Outputs
Structured outputs are a way to format the output of an LLM in a structured manner. This can be useful for tasks that require specific formatting or data extraction.

In [15]:
from pydantic import BaseModel
from agents import Agent

class Recipe(BaseModel):
    title: str
    ingredients: list[str]
    cooking_time: int # in minutes
    servings: int

recipe_agent = Agent(
    name="Recipe Agent", 
    instructions=("You are an agent for creating recipes. You will be given the name of a food and your job"
                  " is to output that as an actual detailed recipe. The cooking time should be in minutes."),
    output_type=Recipe
)

response = await Runner.run(recipe_agent, "Dal chawal")
recipe = response.final_output
print(recipe)

title='Dal Chawal' ingredients=['1 cup toor dal (split pigeon peas)', '1 cup basmati rice', '2 tablespoons ghee or oil', '1 medium onion, chopped', '2 medium tomatoes, chopped', '2 green chilies, slit (optional)', '1 teaspoon cumin seeds', '1 teaspoon turmeric powder', '1 teaspoon red chili powder', '1 teaspoon coriander powder', 'Salt to taste', '3 cups water (for dal)', '2.5 cups water (for rice)', 'Fresh cilantro, chopped (for garnish)', 'Lemon wedges (optional)'] cooking_time=40 servings=4


### Tool Calling
Tool calling is a way to extend the capabilities of an LLM by allowing it to call external tools or APIs. This can be useful for tasks that require access to external data or services.

In [18]:
from agents import Agent, function_tool

@function_tool
def get_weather(city: str) -> str:
    print(f"Getting weather for {city}")
    return "sunny"

@function_tool
def get_temperature(city: str) -> str:
    print(f"Getting temperature for {city}")
    return "70 degrees"

agent = Agent(
    name="Weather Agent",
    instructions="You are the local weather agent. You are given a city and you need to tell the weather and temperature. For any unrelated queries, say I cant help with that.",
    tools=[get_weather, get_temperature]
)

result = await Runner.run(agent, "Dallas")
result.final_output

Getting weather for Dallas
Getting temperature for Dallas


'The weather in Dallas is sunny, with a temperature of 70 degrees.'

In [ ]:
from agents import WebSearchTool
from pydantic import BaseModel

class News(BaseModel):
    title: str
    date: str
    description: str

class NewsArticles(BaseModel):
    articles: list[News]

news_agent = Agent(
    name="News Reporter",
    instructions="You are a news reporter. Your job is to find recent news articles on the internet about US politics.",
    tools=[WebSearchTool()],
    output_type=News
)

result = await Runner.run(news_agent, "find news")
print(result.final_output)

title='Latest U.S. Politics News – September 18, 2025' date='September 18, 2025' description='**Here are the most recent developments in U.S. politics as of September 18, 2025:**\n\n**1. Stopgap Funding Battle Intensifies**\nThe Republican-led U.S. House has advanced a “clean” stopgap funding bill, extending government funding through November 21. This proposal includes $88 million in enhanced security for lawmakers, judges, and Supreme Court justices—$30 million each for the congressional and executive branches and $28 million for justices’ protection ([reuters.com](https://www.reuters.com/world/us/us-house-advances-stopgap-bill-avert-government-shutdown-2025-09-17/?utm_source=openai)). In response, Senate Democratic Leader Chuck Schumer and other Democrats introduced an alternative bill extending funding only through October 31. Their plan includes healthcare provisions—such as reinstating Medicaid funding cuts, extending Affordable Care Act subsidies, restoring $491 million to publi